# Методы машинного обучения – Лабораторная работа №7

# Автокодировщики

Импортируем нужные на этой лабораторной работе библиотеки:

In [ ]:
import numpy as np
import tensorflow.keras as keras
import pandas as pd
import time
import sys

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

import tensorflow as tf

## Наборы данных MNIST и FASHION MNIST 

Используем в материалах этой лабораторной работы два набора данных:

- [MNIST Dataset](http://yann.lecun.com/exdb/mnist/)
- [Fashion MNIST @ Zalando](https://research.zalando.com/welcome/mission/research-projects/fashion-mnist/)

Эти наборы легко импортируются при помощи `keras` ([```tensorflow.keras.datasets```](https://www.tensorflow.org/api_docs/python/tf/keras/datasets)).

In [ ]:
from keras.datasets import mnist

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = \
    mnist.load_data()

In [ ]:
from keras.datasets import fashion_mnist

(fashion_x_train,fashion_y_train), (fashion_x_test,fashion_y_test) = \
    fashion_mnist.load_data()

Как обычно, выполним типичную нормализацию наборов данных:

In [ ]:
mnist_x_train = mnist_x_train.astype('float32') / 255.
mnist_x_test  = mnist_x_test.astype('float32') / 255.

fashion_x_train = fashion_x_train.astype('float32') / 255.
fashion_x_test  = fashion_x_test.astype('float32') / 255.

Создадим варианты наборов данных для обучения сетей MLP:

In [ ]:
mnist_x_train.shape, mnist_x_test.shape

In [ ]:
mnist_x_trainMLP = mnist_x_train.reshape(
    (len(mnist_x_train), np.prod(mnist_x_train.shape[1:])))
mnist_x_testMLP  = mnist_x_test.reshape(
    (len(mnist_x_test), np.prod(mnist_x_test.shape[1:])))
mnist_x_trainMLP.shape, mnist_x_testMLP.shape

In [ ]:
fashion_x_train.shape, fashion_x_test.shape

In [ ]:
fashion_x_trainMLP = fashion_x_train.reshape(
    (len(fashion_x_train), np.prod(fashion_x_train.shape[1:])))
fashion_x_testMLP  = fashion_x_test.reshape(
    (len(fashion_x_test), np.prod(fashion_x_test.shape[1:])))
fashion_x_trainMLP.shape, fashion_x_testMLP.shape

## Функция для создания автокодировщиков MLP

Автокодировщик состоит из двух частей: кодировщика (_encoder_) и декодера (_decoder_). Функция ```create_autoencoders()``` возвращает следующие объекты как отдельные модели:

- кодировщик 
- декодер
- полную модель, включающую кодировщик и декодер.

In [ ]:
def create_autoencoders (feature_layer_dim = 16):
    input_img = Input(shape = (784,), name = 'Input_Layer')
    # Слой encoded имеет размерность, равную feature_layer_dim 
    # и содержит закодированные входные данные 
    encoded = Dense(feature_layer_dim, activation = 'relu', 
                    name = 'Encoded_Features')(input_img)
    decoded = Dense(784, activation = 'sigmoid', 
                    name = 'Decoded_Input')(encoded)

    autoencoder = Model(input_img, decoded)
    encoder = Model(input_img, encoded)

    encoded_input = Input(shape = (feature_layer_dim,))
    decoder = autoencoder.layers[-1]
    decoder = Model(encoded_input, decoder(encoded_input))

    return autoencoder, encoder, decoder

## Автокодировщик со слоями с $(784,16,784)$ нейронами

Начнем с автокодировщика, имеющего слои с размерностями $(784, 16, 784)$.




In [ ]:
autoencoder16, encoder16, decoder16 = create_autoencoders(16)

In [ ]:
keras.utils.plot_model(autoencoder16, show_shapes=True)

Теперь нужно скомпилировать модель и обучить ее на данных. Для работы с автоэнкодерами достаточно простой модели **создать** $\rightarrow$ **compile** $\rightarrow$ **fit**.

In [ ]:
autoencoder16.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
history = autoencoder16.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testMLP, mnist_x_testMLP),
                verbose = 1);

In [ ]:
encoded_imgs = encoder16.predict(mnist_x_testMLP)
decoded_imgs = decoder16.predict(encoded_imgs)

### Относительно сжатия

Поскольку мы сохранили и кодировщик, и декодер, мы можем создать закодированную версию изображений. Обратите внимание, что тип данных изображений — ```float32```. Каждый объект ```float32``` требует 4 байта. Исходные данные в ```mnist_x_test``` имеют вид ```(10000,784)```. Это означает, что пространство, необходимое для хранения этого массива, равно

$$
10000 \textrm{ изображений} \times 784 \textrm{ пикселей}\times 4 \textrm{ байт} = 29 \textrm{ Мб}
$$

Закодированные изображения требуют гораздо меньше места. На самом деле необходимый размер

$$
10000 \textrm{ изображений}\times 16 \textrm{ скрытых признаков}\times 4 \textrm{ байт} = 625 \textrm{ Кб}
$$

поэтому, если мы примем потерю качества в сжатых данных, мы получим невероятную степень сжатия около $46$. Эти размеры легко проверить, сохранив данные в файле и проверив размер файла. Обратите внимание, что размер файла будет больше, так как есть некоторая дополнительная информация, которую необходимо сохранить.

In [ ]:
np.save('temp_orig', mnist_x_testMLP)
! ls -al temp_orig*

In [ ]:
np.save('temp_encoded', encoded_imgs)
! ls -al temp_encoded*

## Анализ реконструированных изображений

Рассмотрим эффект от сжатия входных изображений. Наличие всего 16 нейронов в среднем слое дает следующий результат:

In [ ]:
import matplotlib.pyplot as plt

def image_show(orig_imgs, dec_imgs, fname=None):
    n = 10  # кол-во изображений
    fig = plt.figure(figsize=(20, 4))
    for i in range(n):
        # исходные изображения
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(orig_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # реконструированные изображения
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(dec_imgs[i].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.show()

    if fname:
        fig.savefig(fname)    

In [ ]:
image_show(mnist_x_testMLP, decoded_imgs)

## Автокодировщик со слоями с $(784,64,784)$ нейронами


In [ ]:
autoencoder64, encoder64, decoder64 = create_autoencoders(64)

In [ ]:
autoencoder64.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
autoencoder64.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testMLP, mnist_x_testMLP),
                verbose = 1);

In [ ]:
encoded_imgs = encoder64.predict(mnist_x_testMLP)
decoded_imgs = decoder64.predict(encoded_imgs)

In [ ]:
image_show(mnist_x_testMLP, decoded_imgs)

## Автокодировщик со слоями с $(784,8,784)$ нейронами

При уменьшении количества нейронов в среднем слое качество реконструкции существенно падает:

In [ ]:
autoencoder8, encoder8, decoder8 = create_autoencoders(8)
autoencoder8.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder8.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testMLP, mnist_x_testMLP),
                verbose = 1);

In [ ]:
encoded_imgs = encoder8.predict(mnist_x_testMLP)
decoded_imgs = decoder8.predict(encoded_imgs)

In [ ]:
image_show(mnist_x_testMLP, decoded_imgs)

## Добавление ограничения разреженности

Еще один способ ограничить компактность представлений — добавить ограничение разреженности на активность скрытого представления, чтобы «срабатывало» меньшее количество нейронов. В Keras это можно сделать, добавив параметр `activity_regularizer` в плотный слой:

In [ ]:
from keras import regularizers

encoding_dim = 16

input_img = Input(shape=(784,))

# плотный слой с регуляризатором активности L1 
encoded = Dense(encoding_dim, activation='relu',
    activity_regularizer=regularizers.l1(10e-5))(input_img)

decoded = Dense(784, activation='sigmoid')(encoded)

autoencoderL1 = Model(input_img, decoded)

In [ ]:
autoencoderL1.compile(optimizer='adam', loss='binary_crossentropy')
autoencoderL1.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testMLP, mnist_x_testMLP),
                verbose = 1);

In [ ]:
decoded_imgs = autoencoderL1.predict(mnist_x_testMLP)

In [ ]:
image_show(mnist_x_testMLP, decoded_imgs)

## Глубокий автокодировщик MLP

Вместо одного слоя в кодировщике и декодере можно использовать несколько слоев:

In [ ]:
encoding_dim = 16

input_img = Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)
autoencoderD = Model(input_img, decoded)

In [ ]:
keras.utils.plot_model(autoencoderD, show_shapes=True)

In [ ]:
autoencoderD.compile(optimizer='adam', loss='binary_crossentropy')

autoencoderD.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testMLP, mnist_x_testMLP),
                verbose = 1);

## Сверточный автокодировщик

Поскольку входными данными являются изображения, имеет смысл использовать сверточные нейронные сети в качестве кодировщиков и декодеров. На практике автокодировщики, применяемые к изображениям, всегда являются сверточными автокодировщиками — они работают намного лучше.

Кодировщик будет состоять из стека слоев `Conv2D` и `MaxPooling2D` (максимальный пул используется для пространственной понижающей дискретизации), а декодер будет состоять из стека слоев `Conv2D` и `UpSampling2D`.

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,UpSampling2D

input_img = keras.Input(shape=(28, 28, 1))

x = Conv2D(16, (3,3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(8, (3,3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same')(x)

# Здесь представление (4, 4, 8) т.е. 128-мерное

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoderCNN = Model(input_img, decoded)

In [ ]:
autoencoderCNN.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
mnist_x_train.shape, mnist_x_test.shape

In [ ]:
mnist_x_trainCNN = mnist_x_train.reshape(
    (len(mnist_x_train), 28, 28, 1))
mnist_x_testCNN  = mnist_x_test.reshape(
    (len(mnist_x_test), 28, 28, 1))
mnist_x_trainCNN.shape, mnist_x_testCNN.shape

In [ ]:
autoencoderCNN.fit(mnist_x_trainCNN, mnist_x_trainCNN,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testCNN, mnist_x_testCNN),
                verbose = 1);

In [ ]:
keras.utils.plot_model(autoencoderCNN, show_shapes=True)

In [ ]:
decoded_imgs = autoencoderCNN.predict(mnist_x_testCNN)

In [ ]:
image_show(mnist_x_testCNN, decoded_imgs)

## Классификация по латентным признакам

Проверим, как можно использовать скрытые (латентные) признаки или, другими словами, выходные данные среднего слоя, чтобы выполнить классификацию объектов. Что происходит с точностью и производительностью алгоритмов? Проверим несколько примеров.

### Алгоритм kNN (метод ближайших соседей)

Первым протестируем алгоритм kNN из библиотеки ```sklearn```.

In [ ]:
encoded_train_imgs = encoder8.predict(mnist_x_trainMLP)

Обратите внимание, что переменная ```encoded_train_imgs``` имеет только 8 признаков (последний обученный нами автокодировщик имел только 8 нейронов в среднем слое). В исходном наборе данных было 784 признаков (оттенки серого в пикселях изображений).

In [ ]:
encoded_train_imgs.shape

In [ ]:
def plot_confusion_matrix(data, labels, output_filename):
    """Визуализация матрицы ошибок при помощи heatmap
 
    Аргументы:
        data (список списков): матрица ошибок (confusion matrix)
        labels (список): Метки для осей x и y
        output_filename (текст): Путь к выходному файлу
 
    """
    sns.set(color_codes=True)
    plt.figure(1, figsize=(9, 6))
 
    sns.set(font_scale=1.3)
    ax = sns.heatmap(data, annot=True, cmap="Blues", 
                     cbar_kws={'label': 'Шкала'},fmt='d')
 
    ax.set_xticklabels(labels, fontsize = 16)
    ax.set_yticklabels(labels, fontsize = 16)
 
    ax.set_xlabel("Прогнозные метки", fontsize = 16)
    ax.set_ylabel("Истинные метки", fontsize = 16)
 
    plt.savefig(output_filename, bbox_inches='tight', dpi=300)
    plt.show()
    plt.close()

In [ ]:
# импорт необходимых библиотек 
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
import seaborn as sns

Обучим классификатор на закодированных изображениях и измерим, сколько времени потребуется для обучения. Позже сделаем то же самое с исходным набором данных и сравним результаты и время выполнения.

In [ ]:
start = time.time()
 
# обучаем классификатор KNN 
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=7).fit(encoded_train_imgs, 
                                              mnist_y_train) 
  
# доля верных ответов на X_test 
accuracy = knn.score(encoded_imgs, mnist_y_test) 
print (accuracy )

end = time.time()
print("Время работы",end - start,"сек.") 


Всего с 8 признаками получаем 89% точности за несколько секунд.

In [ ]:
# строим матрицу ошибок 
knn_predictions = knn.predict(encoded_imgs)  

cm = confusion_matrix(mnist_y_test, knn_predictions)

plot_confusion_matrix(cm, [0,1,2,3,4,5,6,7,8,9], 
                      "confusion_matrix.png")

### Алгоритм kNN со всеми признаками

Теперь давайте обучим классификатор со всеми 784 признаками.

In [ ]:
start = time.time()
 
knn2 = KNeighborsClassifier(n_neighbors = 7).fit(
    mnist_x_trainMLP, mnist_y_train) 

# доля верных ответов на X_test 
accuracy = knn2.score(mnist_x_testMLP, mnist_y_test)  
print (accuracy )

end = time.time()
print("Время работы",end - start,"sec.") 

Со всеми признаками получаем точность 96% (на 7% больше, чем с 8 признаками), но это занимает больше времени. Для большого набора данных разница может быть гораздо более существенной.

## Функция потерь MSE

Автокодировщики настолько гибки, что работают, даже если мы используем в качестве функции потерь MSE. 

In [ ]:
dim = 16

input_img = Input(shape = (784,))
encoded = Dense(dim, activation = 'relu')(input_img)
decoded = Dense(784, activation = 'sigmoid')(encoded)

autoencoderMSE = Model(input_img, decoded)
encoderMSE = Model(input_img, encoded)

In [ ]:
encoded_input = Input(shape = (dim,))
decoder = autoencoderMSE.layers[-1]
decoderMSE = Model(encoded_input, decoder(encoded_input))

In [ ]:
autoencoderMSE.compile(optimizer='adam', loss='mse')

In [ ]:
autoencoderMSE.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(mnist_x_testMLP, mnist_x_testMLP),
                verbose = 1);

In [ ]:
encoded_imgs = encoderMSE.predict(mnist_x_testMLP)
decoded_imgs = decoderMSE.predict(encoded_imgs)

In [ ]:
image_show(mnist_x_testMLP, decoded_imgs)

## Набор данных Fashion MNIST

Используем автокодировщик с 8 нейронами в среднем слое с набором данных FASHION MNIST.

In [ ]:
autoencoder8, encoder8, decoder8 = create_autoencoders(8)
autoencoder8.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder8.fit(fashion_x_trainMLP, fashion_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(fashion_x_testMLP, fashion_x_testMLP),
                verbose = 1);

In [ ]:
encoded_imgs = encoder8.predict(fashion_x_testMLP)
decoded_imgs = decoder8.predict(encoded_imgs)

In [ ]:
image_show(fashion_x_testMLP, decoded_imgs)

Снова построим классификатор со всеми данными:

In [ ]:
start = time.time()

knn = KNeighborsClassifier(n_neighbors = 7).fit(
    fashion_x_trainMLP, fashion_y_train) 

# Доля верных ответов на X_test 
accuracy = knn.score(fashion_x_testMLP, fashion_y_test) 
print (accuracy )

end = time.time()
print(end - start)

## Алгоритм kNN на обученном представлении

In [ ]:
encoded_fashion_train_imgs = encoder8.predict(fashion_x_trainMLP)
encoded_fashion_test_imgs = encoder8.predict(fashion_x_testMLP)

In [ ]:
start = time.time()
 
# обучаем классификатор KNN 
knn = KNeighborsClassifier(n_neighbors = 7).fit(
    encoded_fashion_train_imgs, fashion_y_train) 
  
# доля верных ответов на X_test 
accuracy = knn.score(encoded_fashion_test_imgs, fashion_y_test) 
print (accuracy )

end = time.time()
print(end - start)

Опять теряем около 5% доли верных ответов за счет существенного ускорения работы. 

## Доля верных ответов для kNN с автокодировщиком с количеством нейронов $(784,16,784)$

In [ ]:
autoencoder16, encoder16, decoder16 = create_autoencoders(16)
autoencoder16.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder16.fit(fashion_x_trainMLP, fashion_x_trainMLP,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(fashion_x_testMLP, fashion_x_testMLP),
                verbose = 1);

In [ ]:
encoded_fashion_train_imgs = encoder16.predict(fashion_x_trainMLP)
encoded_fashion_test_imgs = encoder16.predict(fashion_x_testMLP)

In [ ]:
start = time.time()
 
# обучение классификатора KNN 
knn = KNeighborsClassifier(n_neighbors = 7).fit(
    encoded_fashion_train_imgs, fashion_y_train) 
  
# доля верных ответов на X_test 
accuracy = knn.score(encoded_fashion_test_imgs, fashion_y_test) 
print (accuracy )

end = time.time()
print(end - start)

Увеличение количества нейронов в среднем слое до 16 увеличивает долю верных ответов при незначительном увеличении времени работы.

## Обнаружение аномалий при помощи автокодировщиков

Разберемся, что означает обнаружение аномалий. Создадим специальный набор данных, состоящий из 10000 изображений тестового набора данных MNIST и одного изображения из набора данных Fashion MNIST. Нашей целью будет найти это изображение автоматически. 

In [ ]:
x_test = np.concatenate((mnist_x_testMLP, 
                         fashion_x_testMLP[0].reshape(1,784)))
x_test.shape

Все изображения в наборе данных MNIST представляют собой написанные от руки цифры. Ниже вы можете увидеть пример одного из них.

In [ ]:
def mnist_show(img):
    plt.gray()
    plt.tick_params(axis = 'x', which = 'both', bottom = False, 
                    top = False, labelbottom = False) 
    plt.tick_params(axis = 'y', which = 'both', left = False, 
                    right = False, labelleft = False)

    plt.imshow(img.reshape(28, 28))    

In [ ]:
mnist_show(mnist_x_test[10]);

Но изображения в Fashion MNIST — это все изображения оттенками серого предметов одежды. В частности, мы добавляем к рукописным цифрам изображение обуви, которое можно увидеть ниже.

In [ ]:
mnist_show(fashion_x_test[0]);

## Автокодировщик со слоями с $(784,64,784)$  нейронами

In [ ]:
autoencoder64, encoder64, decoder64 = create_autoencoders(64)

In [ ]:
keras.utils.plot_model(autoencoder64, show_shapes = True)

In [ ]:
autoencoder64.compile(optimizer = 'adam', loss = 'binary_crossentropy')

In [ ]:
history = autoencoder64.fit(mnist_x_trainMLP, mnist_x_trainMLP,
                          epochs = 30,
                          batch_size = 256,
                          shuffle = True,
                          validation_data = (mnist_x_testMLP, mnist_x_testMLP),
                          verbose = 1)

In [ ]:
encoded_imgs = encoder64.predict(x_test)
decoded_imgs = decoder64.predict(encoded_imgs)

Теперь мы можем рассчитать ошибку реконструкции ($\textrm{RE}^{[j]}$) изображения $j$, просто вычислив

$$
\textrm{RE}^{[j]} = \sum_{i=1}^{784}\frac{(x_i^{[j]}-x_{rec,i}^{[j]})^2}{m}
$$

где $x_i^{[j]}$ — это значение $i^{го}$ пикселя изображения $j$, а сумма вычисляется по всем пикселям.

In [ ]:
RE = ((x_test - decoded_imgs)**2).mean(axis = 1)
RE_original = RE.copy()

Ошибка реконструкции $\textrm{RE}$ для изображения обуви, которое мы добавили, может быть легко выведена, так как это последний элемент вектора ```RE```.

In [ ]:
RE[-1]

Легко видеть, что это самая высокая ошибка реконструкции $\textrm{RE}$, которая у нас есть для всех 10000 изображений. Мы можем проверить это, отсортировав вектор ```RE```.

In [ ]:
RE.shape

In [ ]:
RE.sort()
print(RE[9990:])

Вы можете видеть, что вторая по величине ошибка реконструкции составляет менее половины $\textrm{RE}$ для добавленного изображения. Ниже вы можете увидеть исходное изображение и то, которое реконструировал обученный автокодировщик. Вы можете видеть, что восстановленное изображение совсем не похоже на оригинал.

### Самая большая ошибка реконструкции RE

In [ ]:
biggest_re_pos = np.argmax(RE_original)

In [ ]:
fig = plt.figure(figsize = (14, 7))

ax = fig.add_subplot(1, 2, 1)
plt.title('Исходное изображение', fontsize = 16)
mnist_show(x_test[biggest_re_pos])

ax = fig.add_subplot(1, 2, 2)
plt.title('Реконструированное изображение', fontsize = 16)
mnist_show(decoded_imgs[biggest_re_pos]);

Как вы можете видеть ниже, автокодировщик способен идеально реконструировать рукописные изображения цифр.

### Вторая по величине ошибка реконструкции RE

Изображение, показанное ниже (и его реконструированная версия), имеет вторую по величине ошибку реконструкции. Причина ясна, это изображение совсем не похоже на рукописную цифру! Это может даже считаться выбросом в наборе данных.

In [ ]:
second_biggest_re_pos = list(RE_original).index(RE[-2])

In [ ]:
fig = plt.figure(figsize = (14, 7))

ax = fig.add_subplot(1, 2, 1)
plt.title('Исходное изображение', fontsize = 16)
mnist_show(x_test[second_biggest_re_pos])

ax = fig.add_subplot(1, 2, 2)
plt.title('Реконструированное изображение', fontsize = 16)
mnist_show(decoded_imgs[second_biggest_re_pos]);

### Третья по величине ошибка реконструкции RE

In [ ]:
third_biggest_re_pos = list(RE_original).index(RE[-3])

In [ ]:
fig = plt.figure(figsize = (14, 7))

ax = fig.add_subplot(1, 2, 1)
plt.title('Исходное изображение', fontsize = 16)
mnist_show(x_test[third_biggest_re_pos])

ax = fig.add_subplot(1, 2, 2)
plt.title('Реконструированное изображение', fontsize = 16)
mnist_show(decoded_imgs[third_biggest_re_pos]);

## Подавление шумов изображений с помощью автокодировщиков на основе нейронных сетей MLP

Используем [набор данных MNIST](http://yann.lecun.com/exdb/mnist/). Импортируем его из `keras` при помощи [```tensorflow.keras.datasets```](https://www.tensorflow.org/api_docs/python/tf/keras/datasets).

In [ ]:
# Загружаем набор данных MNIST
(input_train, target_train), (input_test, target_test) = \
    mnist.load_data()

Выполним типовую нормализацию:

In [ ]:
# Переводим числа в тип float32
input_train = input_train.astype('float32')
input_test = input_test.astype('float32')

# Нормализуем данные
input_train = input_train / 255.
input_test = input_test / 255.

In [ ]:
# Изменим форму для сетей MLP
input_trainMLP = input_train.reshape(
    (len(input_train), np.prod(input_train.shape[1:])))
input_testMLP = input_test.reshape(
    (len(input_test), np.prod(input_test.shape[1:])))

In [ ]:
# Изменим форму для сетей CNN
img_width, img_height = 28, 28

input_trainCNN = input_train.reshape(
    input_train.shape[0], img_width, img_height, 1)
input_testCNN = input_test.reshape(
    input_test.shape[0], img_width, img_height, 1)
input_shapeCNN = (img_width, img_height, 1)

Построим пример изображения для каждого возможного класса (то есть цифр от 0 до 9).

In [ ]:
from random import *

def get_random_element_with_label (data, lbls, lbl):
    """Возвращает массив numpy (с одним столбцом) 
    с экземплярами выбранной метки."""

    tmp = lbls == lbl
    subset = data[tmp.flatten(), :]
    return subset[randint(0, subset.shape[0]), :]

In [ ]:
# Изменяем форму обучающего набора данных
input_example = input_train.reshape(60000, 784)

In [ ]:
# Следующий код создает массив numpy, где в столбце 0 будет
# пример метки 0, в столбце 1 метки 1 и так далее.
labels_overview = np.empty([784, 10])
for i in range (0, 10):
    col = get_random_element_with_label(
        input_example, target_train, i)
    labels_overview[:,i] = col

In [ ]:
f = plt.figure(figsize = (15, 15))
count = 1
for i in range(0, 10):
    plt.gray()
    plt.subplot(5, 2, count)
    count = count + 1
    plt.subplots_adjust(hspace = 0.5)
    plt.title('Цифра: ' + str(i))
    some_digit_image = labels_overview[:, i]
    plt.imshow(some_digit_image.reshape(28, 28))
    plt.axis('off')
    pass

## Добавление шума в набор данных

Добавим источник шума к изображениям MNIST (с помощью функции `np.random.normal`), так как нашей целью будет удалить из них этот шум.

In [ ]:
noise_factor = 0.55
pure = input_trainMLP
pure_test = input_testMLP
noise = np.random.normal(0, 1, pure.shape)
noise_test = np.random.normal(0, 1, pure_test.shape)
noisy_input = pure + noise_factor * noise
noisy_input_test = pure_test + noise_factor * noise_test

Теперь нарисуем несколько примеров изображений, искаженных шумом (по одному для каждого класса).

In [ ]:
# Следующий код создает массив numpy, где в столбце 0 будет
# пример метки 0, в столбце 1 метки 1 и так далее.
labels_overview = np.empty([784, 10])
for i in range (0, 10):
    col = get_random_element_with_label(
        noisy_input, target_train, i)
    labels_overview[:,i] = col

In [ ]:
f = plt.figure(figsize = (15, 15))
count = 1
for i in range(0, 10):
    plt.gray()
    plt.subplot(5, 2, count)
    count = count + 1
    plt.subplots_adjust(hspace = 0.5)
    plt.title('Цифра: ' + str(i))
    some_digit_image = labels_overview[:, i]
    plt.imshow(some_digit_image.reshape(28, 28))
    plt.axis('off')
    pass

## Автокодировщик на базе сетей MLP

In [ ]:
autoencoder32, encoder32, decoder32 = create_autoencoders(32) 

In [ ]:
autoencoder32.summary()

In [ ]:
keras.utils.plot_model(autoencoder32, show_shapes = True)

Воспользуемся следующими параметрами для обучения модели:

In [ ]:
batch_size = 150
no_epochs = 30
validation_split = 0.2

In [ ]:
autoencoder32.compile(optimizer = 'adam', loss = 'binary_crossentropy')
autoencoder32.fit(noisy_input, pure,
                epochs = no_epochs,
                batch_size = batch_size,
                validation_split = validation_split);

## Примеры изображений с подавленным шумом

Построим несколько примеров изображений с подавленным шумом, сравнивая их с исходными чистыми изображениями, чтобы увидеть, насколько хорошо работает встроенный автокодировщик.

In [ ]:
# Построим изображения без шума
number_of_visualizations = 6
samples = noisy_input_test[:number_of_visualizations]
targets = target_test[:number_of_visualizations]
denoised_images = autoencoder32.predict(samples)

In [ ]:
# нарисуем изображения без шума
for i in range(0, number_of_visualizations):
    plt.gray()
    # изображение и реконструкция
    noisy_image = noisy_input_test[i].reshape(28, 28)
    pure_image  = pure_test[i].reshape(28, 28)
    denoised_image = denoised_images[i].reshape(28, 28)
    input_class = targets[i]
    # подготовка
    fig, axes = plt.subplots(1, 3)
    fig.set_size_inches(12, 7)
    # выводим изображение и реконструкцию
    axes[0].imshow(noisy_image)
    axes[0].set_title('Изображение с шумом', fontsize = 16)
    axes[0].get_xaxis().set_visible(False)
    axes[0].get_yaxis().set_visible(False)
    axes[1].imshow(pure_image)
    axes[1].set_title('Чистое изображение', fontsize = 16)
    axes[1].get_xaxis().set_visible(False)
    axes[1].get_yaxis().set_visible(False)
    axes[2].imshow(denoised_image)
    axes[2].set_title('Изображение без шума', fontsize = 16)
    axes[2].get_xaxis().set_visible(False)
    axes[2].get_yaxis().set_visible(False)
    plt.show()

Как видно из приведенного выше рисунка, построенная нами модель может реконструировать исходную версию изображений зашумленных цифр. Если у вас есть новые зашумленные изображения того же типа, вы можете применить к ним модель и удалить шум с тех же изображений.

## Автокодировщик с нейронной сетью CNN 

Функция ```create_autoencoder_CNN()``` ниже возвращает модель автокодировщика на базе CNN.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.constraints import max_norm

def create_autoencoder_CNN(max_norm_value = 2.0):
    # Создаем модель
    model = Sequential()
    model.add(Conv2D(64, kernel_size = (3, 3), 
                     kernel_constraint = max_norm(max_norm_value), 
                     activation = 'relu', 
                     kernel_initializer = 'he_uniform', 
                     input_shape = input_shapeCNN))
    model.add(Conv2D(32, kernel_size = (3, 3), 
                     kernel_constraint = max_norm(max_norm_value), 
                     activation = 'relu', 
                     kernel_initializer = 'he_uniform'))
    model.add(Conv2DTranspose(32, kernel_size = (3,3), 
                     kernel_constraint = max_norm(max_norm_value), 
                     activation = 'relu', 
                     kernel_initializer = 'he_uniform'))
    model.add(Conv2DTranspose(64, kernel_size = (3,3), 
                     kernel_constraint = max_norm(max_norm_value), 
                     activation = 'relu', 
                     kernel_initializer = 'he_uniform'))
    model.add(Conv2D(1, kernel_size = (3, 3), 
                     kernel_constraint = max_norm(max_norm_value), 
                     activation = 'sigmoid', padding = 'same'))
    return model

In [ ]:
model = create_autoencoder_CNN()

In [ ]:
keras.utils.plot_model(model, show_shapes = True)

Для сети CNN нужна другая форма входных данных:

In [ ]:
noisy_input = noisy_input.reshape(
    noisy_input.shape[0], img_width, img_height, 1)
pure = pure.reshape(
    pure.shape[0], img_width, img_height, 1)

noisy_input.shape,pure.shape

Используем следующие параметры для обучения модели:

In [ ]:
batch_size = 150
no_epochs = 5
validation_split = 0.2

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy')
model.fit(noisy_input, pure,
          epochs = no_epochs,
          batch_size = batch_size,
          validation_split = validation_split);

## Примеры изображений с подавленным шумом

Построим несколько примеров изображений с подавленным шумом, сравнивая их с исходными чистыми изображениями, чтобы увидеть, насколько хорошо работает встроенный автокодировщик.

In [ ]:
noisy_input_test = noisy_input_test.reshape(
    noisy_input_test.shape[0], img_width, img_height, 1)
pure_test = pure_test.reshape(
    pure_test.shape[0], img_width, img_height, 1)

noisy_input_test.shape, pure_test.shape

In [ ]:
# Построим изображения без шума
number_of_visualizations = 6
samples = noisy_input_test[:number_of_visualizations]
targets = target_test[:number_of_visualizations]
denoised_images = model.predict(samples)

In [ ]:
for i in range(0, number_of_visualizations):
    plt.gray()
    
    noisy_image = noisy_input_test[i][:, :, 0]
    pure_image  = pure_test[i][:, :, 0]
    denoised_image = denoised_images[i][:, :, 0]
    input_class = targets[i]
    
    fig, axes = plt.subplots(1, 3)
    fig.set_size_inches(12, 7)
    
    axes[0].imshow(noisy_image)
    axes[0].set_title('Noisy image', fontsize = 16)
    axes[0].get_xaxis().set_visible(False)
    axes[0].get_yaxis().set_visible(False)
    axes[1].imshow(pure_image)
    axes[1].set_title('Pure image', fontsize = 16)
    axes[1].get_xaxis().set_visible(False)
    axes[1].get_yaxis().set_visible(False)
    axes[2].imshow(denoised_image)
    axes[2].set_title('Denoised image', fontsize = 16)
    axes[2].get_xaxis().set_visible(False)
    axes[2].get_yaxis().set_visible(False)
    fig.suptitle(f'MNIST target = {input_class}')
    plt.show();

Как видно из приведенного выше рисунка, построенная нами модель может реконструировать исходную версию изображений зашумленных цифр. Если у вас есть новые зашумленные изображения того же типа, вы можете применить к ним модель и удалить шум с тех же изображений.

#### Задание (10 баллов)

Для закрепленного за Вами варианта лабораторной работы:

1. Загрузите заданный в индивидуальном задании набор данных с изображениями из Tensorflow Datasets с разбиением на обучающую и тестовую выборки. Оставьте в обучающей и тестовой выборках диапазон классов, указанных в индивидуальном задании. Если изображения цветные (с тремя каналами), то перекодируйте их в одноцветные (оттенки серого).
2. Создайте и обучите на обучающей выборке автокодировщик архитектуры, указанной в индивидуальном задании. Визуализируйте несколько исходных и восстановленных автокодировщиком изображений. 
3. Оцените качество модели автокодировщика на тестовой выборке по показателю, указанному в индивидуальном задании.
4. Оставьте в наборах изображения первых двух классов диапазона, указанного в индивидуальном задании первыми. Визуализируйте набор данных на плоскости, соответствующей двум первым латентным признакам, отображая точки различных классов разными цветами. Подпишите оси и рисунок, создайте легенду для классов набора данных.
5. Выполните бинарную классификацию изображений по латентным (скрытым) признакам и всем признакам при помощи классификатора метода ближайших соседей (kNN). Оцените долю верных ответов (accuracy) для двух построенных классификаторов
6. Визуализируйте ROC-кривые для построенных классификаторов на одном рисунке (с легендой) (Указание: используйте метод predict_proba() класса KNeighborsClassifier).
7. Визуализируйте границы принятия решений классификатора kNN для латентных признаков на плоскости, соответствующей двум первым латентным признакам (для прочих латентных признаков задайте средние/медианные значения). 
8. Определите на первоначальной тестовой выборке изображение, имеющее наибольшую ошибку реконструкции. Выведите для этого изображения первоначальное и реконструированное изображения. 
